In [ ]:
import json
import numpy as np
import pandas as pd
import pyproj
import plotly.graph_objects as go


In [ ]:
# load vehicle detections
car_triangulations_df = pd.read_csv(
    "/home/sf_afn/Downloads/univrses_data/parked_car_triangulations_2023_04_24.csv"
)

# Load the parking segments
with open(
    "/home/sf_afn//Downloads/univrses_data/permitted_parking_linestrings.json"
) as user_file:
    parking_json = json.load(user_file)

In [ ]:
car_triangulations_df.head()


In [ ]:
def convert_crs(triangulations_df):
    """
    convert a point with epsg:4326 to epsg:3011

    """
    transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:3011")

    x_3011 = []
    y_3011 = []

    for idx, row in triangulations_df.iterrows():
        y, x = transformer.transform(row["latitude"], row["longitude"])
        x_3011.append(x)
        y_3011.append(y)

    triangulations_df["x_3011"] = x_3011
    triangulations_df["y_3011"] = y_3011
    triangulations_df["xy_3011"] = triangulations_df.apply(
        lambda row: [row["x_3011"], row["y_3011"]], axis=1
    )

    return triangulations_df


In [ ]:
car_triangulations_df = convert_crs(car_triangulations_df)


In [ ]:
car_triangulations_df.head()


In [ ]:
parking_json


In [ ]:
parking_json.keys()


In [ ]:
for p in parking_json["features"]:
    print(p["geometry"]["coordinates"][0])

In [ ]:
fig = go.Figure()

# plot permitted parking
fig.add_trace(
    go.Scatter(
        x=[
            p[0]
            for f in parking_json["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        y=[
            p[1]
            for f in parking_json["features"]
            for p in f["geometry"]["coordinates"] + [[None, None]]
        ],
        name="permitted_parking",
    )
)

# plot triangulation point
fig.add_trace(
    go.Scatter(
        x=car_triangulations_df["x_3011"],
        y=car_triangulations_df["y_3011"],
        name="car_detected",
        mode="markers",
    )
)
fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig.show()

In [ ]:
# singular triangulation stuff

car_triangulations_df.iloc[[2]]

In [ ]:
car_triangulations_df[:2]


In [ ]:
# def extract_triangulation_xy(triangulation_df):
#     points = []
#     for idx, row in triangulation_df.iterrows():
#         # print(idx, row['x_3011'])
#         point = [row["x_3011"], row["y_3011"]]
#         points.append(point)
#     return point

In [ ]:
# test_point = extract_triangulation_xy(car_triangulations_df.iloc[[42]])
test_point = car_triangulations_df["xy_3011"].iloc[42]
test_point


In [ ]:
def parking_proximity(tri_point, parking_json, radius):
    """
    filter permitted parking within the distance of a particular point
    """
    nearby_prkng_ids = []
    for prkng_idx, p in enumerate(parking_json["features"]):
        for coords in p["geometry"][
            "coordinates"
        ]:  # this is the node on the permitted  parking line?
            distance = np.linalg.norm(np.array(tri_point) - np.array(coords))
            if distance < radius:
                nearby_prkng_ids.append(prkng_idx)
                continue  # doing it for all the coords on pp line

    return nearby_prkng_ids

In [ ]:
nb_p_ids = parking_proximity(test_point, parking_json, radius=20)

In [ ]:
nb_p_ids


In [ ]:
def parking_line_at_index(parking_json, idx):
    parking_line = parking_json["features"][idx]["geometry"]["coordinates"]

    return parking_line

In [ ]:
test_line = parking_line_at_index(parking_json, 2677)
test_line


In [ ]:
fig_1 = go.Figure()

fig_1.add_trace(
    go.Scatter(
        x=[test_line[0][0], test_line[1][0]],
        y=[test_line[0][1], test_line[1][1]],
        name="line",
    )
)

fig_1.add_trace(
    go.Scatter(
        x=[test_point[0]],
        y=[test_point[1]],
        name="pointy",
        mode="markers",
    )
)

fig_1.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig_1.show()

In [ ]:
# ---> BK1

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=[
            p[0]
            for idx in nb_p_ids
            for p in parking_json["features"][idx]["geometry"]["coordinates"]
            + [[None, None]]
        ],
        y=[
            p[1]
            for idx in nb_p_ids
            for p in parking_json["features"][idx]["geometry"]["coordinates"]
            + [[None, None]]
        ],
        name="parking_in_proximity",
    )
)

fig.add_trace(
    go.Scatter(
        x=[test_point[0]],
        y=[test_point[0]],
        name="car_triangulation",
        mode="markers",
    )
)

fig.update_yaxes(
    scaleanchor="x",
    scaleratio=1,
)

fig.show()

In [ ]:
# nearby_
# for each in tri_points:
# nearby_ids = praking_proximity(each,parking_json, radius = 20)

In [ ]:
def project_point_on_line_segment(tri_point, parking_line):
    """Calculate the projection of a point on a line segment using numpy."""
    # Define the two points of the line segment
    p1, p2 = np.array(parking_line)

    # Calculate the vector between the two points
    v = p2 - p1

    # Normalize the vector
    v_norm = v / np.linalg.norm(v)

    # Calculate the vector between the first point of the line segment and the point we want to project
    u = tri_point - p1

    # Calculate the dot product of the normalized vector and the vector we just calculated
    dot_product = np.dot(u, v_norm)

    # Multiply the dot product by the normalized vector
    projection = p1 + dot_product * v_norm

    # Check if the projection is within the line segment
    if np.dot(projection - p1, v) < 0:
        projection = p1
    elif np.dot(projection - p2, v) > 0:
        projection = p2

    return projection


In [ ]:
test_point_projection = project_point_on_line_segment(test_point, test_line)
test_point_projection

In [ ]:
# find all the projections for a point so far

car_triangulations_df


In [ ]:
car_triangulations_df["xy_3011"].unique
